In [1]:
from copy import deepcopy 
import numpy as np
from math import ceil, floor
import pandas as pd

# NUM_MIL = [19]
# NUM_CIV = [20]
# POLICY_USAGE_RATIO = 0.25
# DAYS = 732
CIV_CP = 5
COST_CIV = 10800
COST_INF = 3000
# TOTAL_INF = [84]
# TOTAL_PROVINCE = [14]


#maximum civs per state
max_civ = 20

#max infrastructure level per state
max_inf = 10



In [6]:
#Scenario 1, consider building civ in provinces with uniform state infrastructure level and trying to minimize total construction days
#Fixed variables: 

#starting civ count
starting_civ = 5
#average infrastructure level in these 2 provinces
avg_inf = 4
#target civ to be built
tar_civ = 4

#Independant variables: number of infra to be built

#Dependant variables: total construction days


# max_civ = 20
# cp_modifier = 1+0.1*inf

final_inf_levels = [i for i in range(avg_inf, max_inf+1)]
days_on_inf = [0]*len(final_inf_levels)
days_on_civ = [(tar_civ*COST_CIV)/(starting_civ*CIV_CP*(1+0.1*avg_inf))]*len(final_inf_levels)
total_days = [0]*len(final_inf_levels)



for i in range(1,len(final_inf_levels)):
    days_on_inf[i] = days_on_inf[i-1]+COST_INF/(starting_civ*CIV_CP*(1+0.1*final_inf_levels[i-1]))
    days_on_civ[i] = tar_civ*COST_CIV/(starting_civ*CIV_CP*(1+0.1*final_inf_levels[i]))

total_days = np.add(days_on_inf,days_on_civ)


print(days_on_inf)
print(days_on_civ)
# print(total_days)
print(pd.DataFrame(total_days))
print(pd.DataFrame(np.diff(total_days)))

[0, 85.71428571428571, 165.71428571428572, 240.71428571428572, 311.30252100840335, 377.96918767507003, 441.1270824119121]
[1234.2857142857142, 1152.0, 1080.0, 1016.4705882352939, 960.0, 909.4736842105264, 864.0]
             0
0  1234.285714
1  1237.714286
2  1245.714286
3  1257.184874
4  1271.302521
5  1287.442872
6  1305.127082
           0
0   3.428571
1   8.000000
2  11.470588
3  14.117647
4  16.140351
5  17.684211


In [65]:
#Scenario 2, make table to find best levels infrastructure to be constructed

#Fixed variables:
starting_civ=15

#Independant variables:
# max_const_days

#table axis: 
# starting infrastructure level from 1 to 10
# target civ to be constructed from 1 to 20

#table cell content:
#Given max construction days, cell would be max infrastructure to be constructed where such construction improves overall civ construction time
# which follows such conditions:
#  - If all delta total construction days wrt. delta inf > 0, cell = 0 inf level to be constructed
#  - If all delta total construction days wrt. delta inf < 0, cell = max inf level (10) - initial inf level
#  - Else, cell = max delta inf level to be constructed that does NOT increase delta total const days wrt. delta inf

def calc(max_const_days):
    table = np.zeros([max_inf, max_civ])
    for row in range(0,max_inf):
        for col in range(0, max_civ):
            avg_inf = row+1
            tar_civ = col+1
            
            final_inf_levels = [i for i in range(avg_inf, max_inf+1)]
            days_on_inf = [0]*len(final_inf_levels)
            days_on_civ = [(tar_civ*COST_CIV)/(starting_civ*CIV_CP*(1+0.1*avg_inf))]*len(final_inf_levels)
            total_days = [0]*len(final_inf_levels)



            for i in range(1,len(final_inf_levels)):
                days_on_inf[i] = days_on_inf[i-1]+COST_INF/(starting_civ*CIV_CP*(1+0.1*final_inf_levels[i-1]))
                days_on_civ[i] = tar_civ*COST_CIV/(starting_civ*CIV_CP*(1+0.1*final_inf_levels[i]))

            total_days = np.add(days_on_inf,days_on_civ)
            total_days_change = np.diff(total_days)
            
            total_days_change = [change for change in total_days_change if change <= 0]
            total_days = total_days[:len(total_days_change)+1]
            
#             if tar_civ == 20 and avg_inf == 5:
#                 print(total_days)
#                 print(final_inf_levels)
#                 print(max_const_days)
            
            total_days = [day for day in total_days if day <= max_const_days]
            try:
                min_total_days = min(total_days)
            except ValueError:
                optimal_inf_index = -float("inf")
            else:
                optimal_inf_index = total_days.index(min_total_days)
           
#             if tar_civ == 20 and avg_inf == 5:
#                 print("after max_const_days filter:")
#                 print(total_days)
#                 print(final_inf_levels)
#                 print(max_const_days)
            if optimal_inf_index != -float("inf"):
                table[row,col] = final_inf_levels[optimal_inf_index]-avg_inf
            else:
                table[row, col] = optimal_inf_index
    return table

table = calc(365*2)
df = pd.DataFrame(table, 
             columns = ["%d Civ" % (i+1) for i in range(0,max_civ)], 
             index = ["%d Inf" % (j+1) for j in range(0,max_inf)])
df
# df.loc["5 Inf", ["5 Civ", "6 Civ"]]

,1 Civ,2 Civ,3 Civ,4 Civ,5 Civ,6 Civ,7 Civ,8 Civ,9 Civ,10 Civ,11 Civ,12 Civ,13 Civ,14 Civ,15 Civ,16 Civ,17 Civ,18 Civ,19 Civ,20 Civ
1 Inf,0.0,0.0,0.0,3.0,6.0,6.0,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
2 Inf,0.0,0.0,0.0,2.0,5.0,8.0,2.0,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
3 Inf,0.0,0.0,0.0,1.0,4.0,7.0,4.0,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
4 Inf,0.0,0.0,0.0,0.0,3.0,6.0,6.0,0.0,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
5 Inf,0.0,0.0,0.0,0.0,2.0,5.0,5.0,3.0,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
6 Inf,0.0,0.0,0.0,0.0,1.0,4.0,4.0,4.0,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
7 Inf,0.0,0.0,0.0,0.0,0.0,3.0,3.0,3.0,1.0,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
8 Inf,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
9 Inf,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
10 Inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
